In [ ]:
import os
import json

folder = "../data/evaluation_results/"

for filename in os.listdir(folder):
    filepath = os.path.join(folder, filename)
    
    with open(filepath, "r") as f:
        data = json.load(f)
    
    average_score = sum(data.values()) / len(data)
    
    # Compute ratio of values equal to 1
    count_ones = sum(1 for v in data.values() if v == 1)
    ratio_ones = count_ones / len(data)
    
    print(f"{filename}: avg={average_score:.2f}, ratio_of_ones={ratio_ones:.2%}")


subset_superset_results_gpt-4.1-mini.json: avg=0.23, ratio_of_ones=23.08%
eq_jaccard_similarity_scores.json: avg=0.46, ratio_of_ones=30.57%
subset_superset_results.json: avg=0.23, ratio_of_ones=23.08%
eq_jaccard_similarity_scores_gpt-4.1-mini.json: avg=0.46, ratio_of_ones=30.57%
